In [1]:
#Import dependencies
import pandas as pd
import json
from pprint import pprint as pp
import pymongo
from pymongo import MongoClient
from pandas.io.json import json_normalize

In [2]:
#Set up mongodb connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.pro3news
collection = db.shares

In [3]:
#Loop through Newspaper json files
newspapers = ['Breitbart','Business_Insider','Fox News','Financial Times','MarketWatch','NYTimes','Reuters','The Guardian','WSJ']
times = ['1_1','1_2','1_3','1_4','2_1','2_2','2_3','2_4','3_1','3_2','3_3','3_4','4_1','4_2']
df = pd.DataFrame(columns = ['newspaper','date','url','title','shares'])

for paper in newspapers:
    for time in times:
        with open(paper+'/'+time+'.json') as json_file:
            data = json.load(json_file)            
            #Normalize the json data
            for data in data['articles']['results']:
                df = df.append(pd.DataFrame.from_dict(json_normalize(data), orient='columns'),sort=False)
                
             

In [4]:
#Pull the News Share columns and rename them
news_shares_df = df.loc[:,['source.title','date','url','title','shares.facebook']]
news_shares_df.rename(columns={'source.title':'newspaper',
                                'shares.facebook':'shares'},
                     inplace=True)
#news_shares_df = news_shares_df.reset_index(drop=True)
news_shares_df.head(100)

newspaper        date                                                url  \
0   Breitbart  2020-01-20  https://www.breitbart.com/asia/2020/01/20/chin...   
0   Breitbart  2020-01-26  https://www.breitbart.com/politics/2020/01/26/...   
0   Breitbart  2020-01-23  https://www.breitbart.com/asia/2020/01/23/chin...   
0   Breitbart  2020-01-22  https://www.breitbart.com/border/2020/01/22/ex...   
0   Breitbart  2020-01-22  https://www.breitbart.com/border/2020/01/22/po...   
..        ...         ...                                                ...   
0   Breitbart  2020-02-12  https://www.breitbart.com/asia/2020/02/12/chin...   
0   Breitbart  2020-02-12  https://www.breitbart.com/national-security/20...   
0   Breitbart  2020-02-10  https://www.breitbart.com/europe/2020/02/10/uk...   
0   Breitbart  2020-02-11  https://www.breitbart.com/health/2020/02/10/co...   
0   Breitbart  2020-02-11  https://www.breitbart.com/national-security/20...   

                                                title  shares  
0   China Says 'No Need to Panic' as Deadly Virus ...   828.0  
0   Arizona Resident is Fifth Coronavirus Case in ...   736.0  
0   China Lockdown Extends to Five Cities, 20 Mill...   448.0  
0   EXCLUSIVE: Chinese Coronavirus 'Really Serious...   351.0  
0   Possible Case of Chinese Coronavirus Reaches M...   321.0  
..                                                ...     ...  
0   China Cancels Formula 1 Grand Prix Amidst Coro...     NaN  
0   Chinese Military Bullies Taiwan in Response to...     NaN  
0   UK: Coronavirus 'Imminent Threat', Activates E...     NaN  
0   Coronavirus: Texans Quarantined in Homes After...     NaN  
0   China: The West Is 'Overreacting' to Coronavir...     NaN  

[100 rows x 5 columns]

In [7]:
news_shares_df = news_shares_df.to_dict('records')

In [9]:
#Add the data to mongodb
collection.insert_many(news_shares_df)